In [1]:
import pandas as pd

In [2]:
import regex as re

In [3]:
from random import sample

In [4]:
df = pd.read_pickle('STAGE_02_18K_quiz.pkl')

In [5]:
df = df.sample(frac=1).reset_index(drop=True)

In [28]:
df_nouns_choices = pd.read_pickle('NOUN_CHOICES.pkl')
df_nouns_choices['NOUN'] = df_nouns_choices['NOUN'].apply(lambda x: x.lower())
df_nouns_choices['CHOICES'] = df_nouns_choices['CHOICES'].apply(lambda x: x[:10])
df_nouns_choices['CHOICES'] = df_nouns_choices['CHOICES'].apply(lambda x: [i for i in x if ',' not in i])
df_nouns_choices.to_pickle('NOUN_CHOICES.pkl')


df_verbs_choices = pd.read_pickle('VERB_CHOICES.pkl')
df_verbs_choices['VERB'] = df_verbs_choices['VERB'].apply(lambda x: x.lower())
df_verbs_choices['CHOICES'] = df_verbs_choices['CHOICES'].apply(lambda x: x[:10])
df_verbs_choices['CHOICES'] = df_verbs_choices['CHOICES'].apply(lambda x: [i for i in x if ',' not in i])
df_verbs_choices.to_pickle('VERB_CHOICES.pkl')


In [7]:
dict_nouns_choices = dict(df_nouns_choices.values.tolist())
dict_verbs_choices = dict(df_verbs_choices.values.tolist())


In [8]:
df

,QUESTION,PAT_TEXT,PAT_LEMMA,EN_LEN,AR_LEN,ar,en,VERB,VERB_LEMMA,NOUN,NOUN_LEMMA
0,"Colonel Yusuf, who had previously been the _ _...",elected president,elect president,23,21,والعقيد يوسف، الذي كان قد انتخب فيما سبق رئيسا...,"Colonel Yusuf, who had previously been the ele...",elected,elect,president,president
1,We consider that the top priority for the Conf...,start negotiations,start negotiation,24,25,ونعتبر أنه ينبغي للمؤتمر أن يولي أولوية عليا ل...,We consider that the top priority for the Conf...,start,start,negotiations,negotiation
2,_ _ _ _ _ the size and gravity of these proble...,Given the size,give the size,21,20,9 - ونظرا لحجم وخطورة هذه المشاكل والتحديات، ف...,Given the size and gravity of these problems a...,Given,give,size,size
3,We will identify synergies with other instrume...,strengthen cooperation,strengthen cooperation,23,18,وسنحدد أوجه التآزر مع صكوك القانون الإنساني ال...,We will identify synergies with other instrume...,strengthen,strengthen,cooperation,cooperation
4,They _ _ _ _ an overview of the activities of ...,gave an overview,give an overview,15,10,وقدما لمحة عن أنشطة لجنة الاتحاد الأفريقي المع...,They gave an overview of the activities of the...,gave,give,overview,overview
...,...,...,...,...,...,...,...,...,...,...,...
18575,"Nonetheless, much remains to be _ _ _ _ in ter...",done in terms,do in term,24,25,وبالرغم من ذلك، هنالك الكثير مما يجب عمله من ح...,"Nonetheless, much remains to be done in terms ...",done,do,terms,term
18576,Today is a great opportunity for the United Na...,find solutions,find solution,25,19,ويشكل اليوم فرصة كبيرة للأمم المتحدة وللحكومات...,Today is a great opportunity for the United Na...,find,find,solutions,solution
18577,The Agency has _ _ _ _ _ _ this issue with the...,raised this issue,raise this issue,20,15,وأثارت الوكالة هذه المسألة مع وزارة الخارجية ف...,The Agency has raised this issue with the Mini...,raised,raise,issue,issue
18578,34. Monaco _ _ _ no plans to decriminalize def...,has no plans,have no plan,25,24,34- لا تعتزم إمارة موناكو إلغاء تجريم التشهير ...,34. Monaco has no plans to decriminalize defam...,has,have,plans,plan


In [9]:
df['QUESTION'] = df.apply(lambda x: f"{x['QUESTION']}\nـ {x['ar']} ـ", axis=1)

In [10]:
print(df['QUESTION'].iloc[3])

We will identify synergies with other instruments of international humanitarian law and _ _ _ _ _ _ _ _ _ _ cooperation with international organisations and civil society in their implementation.
ـ وسنحدد أوجه التآزر مع صكوك القانون الإنساني الدولي الأخرى وسنعزز التعاون مع المنظمات الدولية والمجتمع المدني في تنفيذها. ـ


In [11]:
quiz_verbs = df['QUESTION VERB'.split()]


In [12]:
quiz_verbs

,QUESTION,VERB
0,"Colonel Yusuf, who had previously been the _ _...",elected
1,We consider that the top priority for the Conf...,start
2,_ _ _ _ _ the size and gravity of these proble...,Given
3,We will identify synergies with other instrume...,strengthen
4,They _ _ _ _ an overview of the activities of ...,gave
...,...,...
18575,"Nonetheless, much remains to be _ _ _ _ in ter...",done
18576,Today is a great opportunity for the United Na...,find
18577,The Agency has _ _ _ _ _ _ this issue with the...,raised
18578,34. Monaco _ _ _ no plans to decriminalize def...,has


In [13]:
def create_choices(word, pos= 'verb / noun?' , size=4):
    word = word.lower()
    if pos.lower() == 'noun':
        choices = sample(dict_nouns_choices[word], size)
        return choices
    else:
        choices = sample(dict_verbs_choices[word], size)
        return choices

In [14]:
quiz_verbs['CHOICES'] = quiz_verbs['VERB'].apply(lambda x: create_choices(x, pos='verb'))


<ipython-input-14-13a0834d166b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quiz_verbs['CHOICES'] = quiz_verbs['VERB'].apply(lambda x: create_choices(x, pos='verb'))


In [15]:
quiz_verbs

,QUESTION,VERB,CHOICES
0,"Colonel Yusuf, who had previously been the _ _...",elected,"[rejected, released, educated, affected]"
1,We consider that the top priority for the Conf...,start,"[separated, stay, target, sat]"
2,_ _ _ _ _ the size and gravity of these proble...,Given,"[guided, listen, gotten, lived]"
3,We will identify synergies with other instrume...,strengthen,"[stressed, started, engages, stated]"
4,They _ _ _ _ an overview of the activities of ...,gave,"[wage, save, pave, take]"
...,...,...,...
18575,"Nonetheless, much remains to be _ _ _ _ in ter...",done,"[gone, denied, come, lose]"
18576,Today is a great opportunity for the United Na...,find,"[fled, fill, fired, send]"
18577,The Agency has _ _ _ _ _ _ this issue with the...,raised,"[trained, caused, rise, passed]"
18578,34. Monaco _ _ _ no plans to decriminalize def...,has,"[hits, changes, pays, shares]"


In [16]:
quiz_verbs['Option 1'] = quiz_verbs['CHOICES'].apply(lambda x: x[0])
quiz_verbs['Option 2'] = quiz_verbs['CHOICES'].apply(lambda x: x[1])
quiz_verbs['Option 3'] = quiz_verbs['CHOICES'].apply(lambda x: x[2])
quiz_verbs['Option 4'] = quiz_verbs['CHOICES'].apply(lambda x: x[3])
quiz_verbs['Option 5'] = quiz_verbs['VERB']

quiz_verbs['Correct Answer'] = "5"
quiz_verbs['Time in seconds'] = "20"
quiz_verbs['Image Link'] = ""
quiz_verbs['Question Type'] = "Multiple Choice"

<ipython-input-16-88ec9a90378f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quiz_verbs['Option 1'] = quiz_verbs['CHOICES'].apply(lambda x: x[0])
<ipython-input-16-88ec9a90378f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quiz_verbs['Option 2'] = quiz_verbs['CHOICES'].apply(lambda x: x[1])
<ipython-input-16-88ec9a90378f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [17]:
quiz_verbs = quiz_verbs.rename(columns={'QUESTION': 'Question Text'})

In [18]:
list(quiz_verbs.columns)

['Question Text',
 'VERB',
 'CHOICES',
 'Option 1',
 'Option 2',
 'Option 3',
 'Option 4',
 'Option 5',
 'Correct Answer',
 'Time in seconds',
 'Image Link',
 'Question Type']

In [19]:
quiz_col = ['Question Text',
    'Question Type',
 'Option 1',
 'Option 2',
 'Option 3',
 'Option 4',
'Option 5',
 'Correct Answer',
 'Time in seconds',
 'Image Link',
]

In [20]:
quiz_verbs = quiz_verbs[quiz_col]

In [21]:
str.islower

<method 'islower' of 'str' objects>

In [22]:
def correct_caset(correct, incorrect):
    if correct.islower():
        return incorrect.lower()
    elif correct.istitle():
        return incorrect.title()
    else:
        return incorrect.lower()

In [23]:
quiz_verbs['Option 1'] = quiz_verbs.apply(lambda x: correct_caset(x['Option 5'], x['Option 1']), axis=1)
quiz_verbs['Option 2'] = quiz_verbs.apply(lambda x: correct_caset(x['Option 5'], x['Option 2']), axis=1)
quiz_verbs['Option 3'] = quiz_verbs.apply(lambda x: correct_caset(x['Option 5'], x['Option 3']), axis=1)
quiz_verbs['Option 4'] = quiz_verbs.apply(lambda x: correct_caset(x['Option 5'], x['Option 4']), axis=1)

In [24]:
quiz_verbs

,Question Text,Question Type,Option 1,Option 2,Option 3,Option 4,Option 5,Correct Answer,Time in seconds,Image Link
0,"Colonel Yusuf, who had previously been the _ _...",Multiple Choice,rejected,released,educated,affected,elected,5,15,
1,We consider that the top priority for the Conf...,Multiple Choice,separated,stay,target,sat,start,5,15,
2,_ _ _ _ _ the size and gravity of these proble...,Multiple Choice,Guided,Listen,Gotten,Lived,Given,5,15,
3,We will identify synergies with other instrume...,Multiple Choice,stressed,started,engages,stated,strengthen,5,15,
4,They _ _ _ _ an overview of the activities of ...,Multiple Choice,wage,save,pave,take,gave,5,15,
...,...,...,...,...,...,...,...,...,...,...
18575,"Nonetheless, much remains to be _ _ _ _ in ter...",Multiple Choice,gone,denied,come,lose,done,5,15,
18576,Today is a great opportunity for the United Na...,Multiple Choice,fled,fill,fired,send,find,5,15,
18577,The Agency has _ _ _ _ _ _ this issue with the...,Multiple Choice,trained,caused,rise,passed,raised,5,15,
18578,34. Monaco _ _ _ no plans to decriminalize def...,Multiple Choice,hits,changes,pays,shares,has,5,15,


In [25]:
quiz_verbs.to_pickle('QUIZZIZ_18K_QUIZ_BANK_MULTIPLE_CHOICE_AR_TRANS_ADDED.pkl', protocol=4)

In [27]:
quiz_verbs

,Question Text,Question Type,Option 1,Option 2,Option 3,Option 4,Option 5,Correct Answer,Time in seconds,Image Link
0,"Colonel Yusuf, who had previously been the _ _...",Multiple Choice,rejected,released,educated,affected,elected,5,15,
1,We consider that the top priority for the Conf...,Multiple Choice,separated,stay,target,sat,start,5,15,
2,_ _ _ _ _ the size and gravity of these proble...,Multiple Choice,Guided,Listen,Gotten,Lived,Given,5,15,
3,We will identify synergies with other instrume...,Multiple Choice,stressed,started,engages,stated,strengthen,5,15,
4,They _ _ _ _ an overview of the activities of ...,Multiple Choice,wage,save,pave,take,gave,5,15,
...,...,...,...,...,...,...,...,...,...,...
18575,"Nonetheless, much remains to be _ _ _ _ in ter...",Multiple Choice,gone,denied,come,lose,done,5,15,
18576,Today is a great opportunity for the United Na...,Multiple Choice,fled,fill,fired,send,find,5,15,
18577,The Agency has _ _ _ _ _ _ this issue with the...,Multiple Choice,trained,caused,rise,passed,raised,5,15,
18578,34. Monaco _ _ _ no plans to decriminalize def...,Multiple Choice,hits,changes,pays,shares,has,5,15,


In [26]:
quiz_verb_01 = quiz_verbs.sample(30).drop_duplicates(['Option 5'])


In [ ]:
quiz_verb_01.to_excel('QUIZZIZ_01.xlsx', encoding='utf-8', index=False)

In [ ]:
df_verbs_choices[df_verbs_choices['CHOICES'].apply(lambda x: len([i for i in x if ',' in i])>0)]